In [61]:
import numpy as np
import pandas as pd

In [87]:
def get_mae(model):
    train_pred = model.predict(x_w2v_train)
    score = mean_absolute_error(y_train,train_pred)
    print(model.__class__.__name__, " train MAE: ", np.round(score,3))
    pred = model.predict(x_w2v_test)
    score = mean_absolute_error(y_test,pred)
    print(model.__class__.__name__, " MAE: ", np.round(score,3))

In [88]:
def get_maes(models):
    scores = []
    for model in models:
        score = get_mae(model)
        # scores.append(score)
    # return scores

In [89]:
def get_r2score(model):
    train_pred = model.predict(x_w2v_train)
    pred = model.predict(x_w2v_test)
    train_score = r2_score(y_train,train_pred)
    score = r2_score(y_test,pred)
    print(model.__class__.__name__, " train_R2SCORE: ", np.round(train_score,3))
    print(model.__class__.__name__, " R2SCORE: ", np.round(score,3))

In [90]:
def get_r2scores(models):
    scores = []
    for model in models:
        score = get_r2score(model)
        # scores.append(score)
    # return scores

In [91]:
def get_rmses(models):
    rmses = []
    for model in models:
        rmse = get_rmse(model)
        # rmses.append(rmse)
    # return rmses

In [92]:
def get_rmse(model):
    train_pred = model.predict(x_w2v_train)
    train_mse = mean_squared_error(y_train,train_pred)
    rmse = np.sqrt(train_mse)
    print(model.__class__.__name__, 'train RMSE: ', np.round(rmse,3))
    pred = model.predict(x_w2v_test)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    print(model.__class__.__name__, ' RMSE: ', np.round(rmse,3))
    return rmse

In [68]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rc

In [69]:
def fv_preprocessing(fv):
  fv_df = fv
  afv = fv_df['average_salary']
  tfv = fv_df['total_sale']

  for i, val in enumerate(afv):
    if type(val) == str:
        if '만원' in val:
          tmp = int(val[:-2].replace(',',''))
          fv_df.loc[i,'average_salary'] = tmp
        if '회사' in val:
          fv_df.loc[i,'average_salary'] = np.NAN
        if '수집' in val:
          fv_df.loc[i,'average_salary'] = np.NAN

  for i, val in enumerate(tfv):
    if type(val) == str:
        if '조원' in val:
          tmp = float(val[:-2].replace(',',''))*1000
          fv_df.loc[i,'total_sale'] = tmp
        if '억원' in val:
          tmp = float(val[:-2].replace(',',''))
          fv_df.loc[i,'total_sale'] = tmp
        if '억' in val:
          tmp = float(val[:-1].replace(',',''))
          fv_df.loc[i,'total_sale'] = tmp
          print(fv_df.loc[i,'total_sale'])
          print(i)
        if '회사' in val:
          fv_df.loc[i,'total_sale'] = np.NAN

  return fv_df

In [70]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [71]:
# 데이터 정규화 과정 겪은 후에 contents가 길이 0인 것은 제거
def preprocessing(review):
    #re
    review = re.sub("[^a-zA-Z]", " ", review)
    word_li = review.lower().split()
    sentence = ' '.join(word_li)

    # lemmatization (spacy)
    token_li = nlp(sentence)
    word_li = [token.lemma_ for token in token_li if not token.is_stop]

    # 문자열로 변환 (띄어쓰기 한 칸 기준으로)
    sentence = ' '.join(word_li)

    return sentence

In [72]:
combined_data = pd.read_csv("outlier_remove_data_tf_idf.csv")

In [73]:
error_firm = ['동원홈푸드','휠라홀딩스','트리','와디즈','키위컴퍼니','줌인터넷','시선인터내셔널','브이티코스메틱','유니슨이테크','씨엠비대전방송','서울비젼','더메인즈','조은시스템']
for i in error_firm:
    combined_data = combined_data[combined_data.company_name != i]
combined_data.dropna(axis=0, inplace=True)
combined_data = combined_data.drop(['Unnamed: 0'], axis=1)
combined_data = fv_preprocessing(combined_data)
tor = combined_data['turn_over_rate']
combined_data.drop('turn_over_rate', axis=1, inplace=True)

In [74]:
combined_data = combined_data.drop(['company_name'], axis = 1, inplace = False)

In [75]:
combined_data = combined_data.iloc[:, 2:]

In [76]:
from sklearn.model_selection import train_test_split
x_train, x_test, train_target, test_target = train_test_split(combined_data, tor, test_size=0.2)

In [77]:
x_train['adv_dadv_combined'] = x_train['adv'] + ' ' + x_train['dadv']
x_test['adv_dadv_combined'] = x_test['adv'] + ' ' + x_test['dadv']

In [78]:
import re

##Word2Vec

In [79]:
# Word2Vec은 단어로 이루어진 리스트를 입력값으로 넣어야 한다.
x_train_li = [sentence.split() for sentence in list(x_train['adv_dadv_combined'])]
x_test_li = [sentence.split() for sentence in list(x_test['adv_dadv_combined'])]

In [80]:
!pip install gensim

In [81]:
from gensim.models import word2vec
model = word2vec.Word2Vec(x_train_li, vector_size=500, min_count=10, window=10)
model.save("./w2v_model")

In [82]:
def w2v_features(words, model, num_features):
  # w2v은 단어 별 벡터의 표현이 나오기 때문에, 문장을 표현해 줄때는, 단어 별 벡터의 합을 개수로 나눠서 표현
  # 미리 정해준 feature 차원으로 np.zeros를 만들어 준다.
  feature_vector = np.zeros((num_features), dtype=np.float32)

  count = 0
  # 어휘사전 준비
  index2word_set = set(model.wv.index_to_key)

  for word in words:
    if word in index2word_set:
      count += 1
      # 사전에 해당하는 단어에 대한 던어 벡터를 더함
      feature_vector = np.add(feature_vector, model.wv[word])

  if count == 0:
    feature_vector = np.zeros((num_features), dtype=np.float32)
  else:
    feature_vector = np.divide(feature_vector, count)

  return feature_vector

In [83]:
def get_features(reviews, model, num_features):
  dataset = [w2v_features(sentence, model, num_features) for sentence in reviews]
  review_features_vecs = np.stack(dataset)

  return review_features_vecs

In [84]:
# train
x_w2v_train = get_features(x_train_li, model, num_features=500)
y_train = train_target.to_numpy()

# test
x_w2v_test = get_features(x_test_li, model, num_features=500)
y_test = test_target.to_numpy()

In [85]:
y_test

array([0.45, 0.83, 0.27, 0.31, 0.15, 0.55, 0.09, 0.35, 0.39, 0.2 , 0.63,
       0.18, 0.67, 0.14, 0.2 , 0.33, 0.31, 0.2 , 0.11, 0.34, 0.44, 0.17,
       0.12, 0.21, 0.31, 0.26, 0.18, 0.38, 0.16, 0.17, 0.3 , 0.55, 0.23,
       0.1 , 0.28, 0.21, 0.64, 0.51, 0.22, 0.44, 0.35, 0.23, 0.2 , 0.11,
       0.12, 0.32, 0.09, 0.14, 0.18, 0.16, 0.22, 0.67, 0.3 , 0.18, 0.54,
       0.08, 0.28, 0.13, 0.69, 0.33, 0.22, 0.54, 0.26, 0.39, 0.39, 0.16,
       0.14, 0.18, 0.27, 0.58, 0.18, 0.43, 0.58, 0.1 , 0.18, 0.26, 0.22,
       0.25, 0.82, 0.15, 0.45, 0.19, 0.75, 0.11, 0.32, 0.13, 0.21, 0.35,
       0.21, 0.08, 0.13, 0.44, 0.25, 0.8 , 0.08, 0.79, 0.62, 0.12, 0.29,
       0.8 , 0.23, 0.14, 0.4 , 0.54, 0.44, 0.07, 0.13, 0.14, 0.08, 0.06,
       0.89, 0.06, 0.36, 0.3 , 0.45, 0.08, 0.21, 0.09, 0.32, 0.31, 0.38,
       0.62, 0.39, 0.18, 0.19, 0.28, 0.13, 0.17, 0.27, 0.06, 0.24, 0.09,
       0.07, 0.48, 0.22, 0.53, 0.18, 0.31, 0.15, 0.67, 0.15, 0.18, 0.17,
       0.47, 0.15, 0.53, 0.08, 0.18, 0.36, 0.56, 0.

In [93]:
# Linear Regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


lr_reg = LinearRegression()
lr_reg.fit(x_w2v_train, y_train)

ridge_reg = Ridge()
ridge_reg.fit(x_w2v_train, y_train)

lasso_reg = Lasso()
lasso_reg.fit(x_w2v_train,y_train)

models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)
print('\n')
get_r2scores(models)
print('\n')
get_maes(models)

LinearRegression train RMSE:  0.125
LinearRegression  RMSE:  0.177
Ridge train RMSE:  0.149
Ridge  RMSE:  0.162
Lasso train RMSE:  0.187
Lasso  RMSE:  0.186


LinearRegression  train_R2SCORE:  0.555
LinearRegression  R2SCORE:  0.091
Ridge  train_R2SCORE:  0.367
Ridge  R2SCORE:  0.234
Lasso  train_R2SCORE:  -0.0
Lasso  R2SCORE:  -0.003


LinearRegression  train MAE:  0.094
LinearRegression  MAE:  0.131
Ridge  train MAE:  0.112
Ridge  MAE:  0.121
Lasso  train MAE:  0.148
Lasso  MAE:  0.149
